# single site analysis  

This notebook unites the approaches from the following notebooks:
- Tutorial_Shapefile.ipynb
- Tutorial_Cubo.ipynb

In [ ]:
# libraries
import xarray as xr
import rioxarray
import geopandas as gpd
import rasterio.features
import numpy as np
import cubo
from sen2nbar.nbar import nbar_cubo
import spyndex
from PIL import Image

# plotting
import matplotlib.pyplot as plt

In [ ]:
# load the shapefile which has many polygons. Explicitely use utf8 encoding
sites = gpd.read_file('data/AreaSites/AreaSites.shp', encoding='utf-8')
testsite = 2
# getting polygon data by index
site = sites.loc[[testsite]]

#Find centroid in lat/lon in WGS84 (EPSG:4326)
centroid = site.centroid.to_crs(epsg=4326).iloc[0]
center_x, center_y = centroid.x, centroid.y

# First, find out the UTM zone for the centroid and set CRS to the appropriate UTM zone of the center point
utm_zone = int((centroid.x + 180) // 6) + 1
utm_crs = f'epsg:326{utm_zone}' if centroid.y >= 0 else f'epsg:327{utm_zone}'

# Project the GeoDataFrame to UTM CRS
site_utm = site.to_crs(utm_crs)

# Get the bounding box in the UTM CRS
bounds_utm = site_utm.geometry.total_bounds
min_x, min_y, max_x, max_y = bounds_utm

# Calculate width and height of the bounding box
width = max_x - min_x
height = max_y - min_y
edge_size = max(width, height)

print(f'Center point: {center_x}, {center_y}, Edge size: {edge_size}')

In [ ]:
try:
    data_sentinel2 = cubo.create(
        lat = center_y, lon = center_x,
        collection = "sentinel-2-l2a",
        # Some bands are not processed by nbar_cubo, so we don't need to load them
        # bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"],
        #bands = ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12"],
        bands = ["B02", "B03", "B04"],
        start_date = "2020-05-01",
        end_date = "2020-06-20",
        edge_size = edge_size/10+10, # 1 pixel is 10 m. so we need to convert meters to pixels. adding some extra
        #units="m",
        resolution = 10,
        query = {"eo:cloud_cover": {"lt": 10}}
    )
    
    print("data found:")
    print("  size: ", data_sentinel2.nbytes / 1e6, "MB")
    print("  time: ", len(data_sentinel2.time), "steps")
    
    data_sentinel2 = nbar_cubo(data_sentinel2)
    
except Exception as e:
    print(f"Error: {e}")
    data_sentinel2 = None

In [ ]:
if data_sentinel2.rio.crs is None:
    data_sentinel2.rio.write_crs('EPSG:'+str(data_sentinel2.attrs['epsg']), inplace=True)

if len(set(data_sentinel2['time'].values)) != len(data_sentinel2['time'].values):
  data_sentinel2 = data_sentinel2.groupby("time")
  data_sentinel2 = data_sentinel2.mean(dim="time", skipna=True)

data_sentinel2.load()
print('Data loaded')

In [ ]:
data_sentinel2_subset = data_sentinel2.isel(time=0)
data_sentinel2_subset = data_sentinel2_subset.drop_vars(['cubo:distance_from_center'])
data_sentinel2_subset_reprojected = data_sentinel2_subset.rio.reproject('EPSG:3035', nodata=np.nan)
fig, ax = plt.subplots(1,2, figsize=(10,5))
data_sentinel2_subset.sel(band=["B04", "B03", "B02"]).plot.imshow(robust=True, ax=ax[0])
ax[0].set_title(f'CUBO data in {data_sentinel2_subset.rio.crs}')
data_sentinel2_subset_reprojected.sel(band=["B04", "B03", "B02"]).plot.imshow(robust=True, ax=ax[1])
ax[1].set_title(f'CUBO data in {data_sentinel2_subset_reprojected.rio.crs}')

siteS2 = data_sentinel2_subset

In [ ]:
deadwood = xr.open_zarr('data/MD.zarr')
deadwood = deadwood.rio.write_crs('EPSG:3035')

bounds = site.total_bounds
min_x, min_y, max_x, max_y = bounds
deadwood_subset = deadwood.sel(x=slice(min_x, max_x), y=slice(min_y, max_y))

In [ ]:
dw = deadwood_subset.deadwood.isel(time=0).transpose('y', 'x')

fig, ax = plt.subplots(1,2, figsize=(10,5))
dw.plot(add_colorbar=False, ax=ax[0])
ax[0].set_title(f'Deadwood data in {dw.rio.crs}')
dw_reprojected = dw.rio.reproject('EPSG:32633')
dw_reprojected.plot(add_colorbar=False, ax=ax[1])
ax[1].set_title(f'Deadwood data in {dw_reprojected.rio.crs}')

siteDeadwood = dw_reprojected

In [ ]:
grid_xmin = round(min(siteDeadwood.x.values.min(), siteS2.x.values.min()))
grid_xmax = round(max(siteDeadwood.x.values.max(), siteS2.x.values.max()))
grid_ymin = round(min(siteDeadwood.y.values.min(), siteS2.y.values.min()))
grid_ymax = round(max(siteDeadwood.y.values.max(), siteS2.y.values.max()))
# amount of values
numValues_x = round(np.mean([len(siteDeadwood.x.values), len(siteS2.x.values)]))
numValues_y = round(np.mean([len(siteDeadwood.y.values), len(siteS2.y.values)]))

new_y = np.linspace(grid_ymin, grid_ymax, numValues_y)
new_x = np.linspace(grid_xmin, grid_xmax, numValues_x)

siteS2_regridded = siteS2.interp(y=new_y, x=new_x, method='linear')
siteDeadwood_regridded = siteDeadwood.interp(y=new_y, x=new_x, method='linear')

site = site.to_crs(utm_crs)

print(site.crs)
print(siteDeadwood_regridded.rio.crs)
print(siteS2_regridded.rio.crs)

print('same grid, same System finally')

In [ ]:
mask_S2 = rasterio.features.geometry_mask(
    geometries = site.geometry,
    out_shape = (siteS2_regridded.y.size, siteS2_regridded.x.size),
    transform = siteS2_regridded.rio.transform(),
    invert = True
)

mask_Deadwood = rasterio.features.geometry_mask(
    geometries = site.geometry,
    out_shape = (siteDeadwood_regridded.y.size, siteDeadwood_regridded.x.size),
    transform = siteDeadwood_regridded.rio.transform(),
    invert = False
)

finalS2 = siteS2_regridded.where(mask_S2, np.nan)
finalDeadwood = siteDeadwood_regridded.where(mask_S2, np.nan)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))
finalDeadwood.plot(robust=True, add_colorbar=False, ax=ax[0])
ax[0].set_title('Deadwood')
finalS2.sel(band=["B04", "B03", "B02"]).plot.imshow(robust=True, ax=ax[1])
ax[1].set_title('Sentinel-2')